In [1]:
import rdflib

In [2]:
from rdflib import Graph

In [5]:
g=Graph()

g.parse("rdflibtest.rdf")

<Graph identifier=N45ce7059c6a4435ba6ed8e3bf849a125 (<class 'rdflib.graph.Graph'>)>

# Get all stations :

In [11]:
query_test="""
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX ns: <http://www.owl-ontologies.com/unnamed.owl#>

SELECT * WHERE {
  ?poi rdf:type ns:Station.
  ?poi ns:hasLatitude ?lat.
  ?poi ns:hasLongitude ?long.
  ?poi ns:hasName ?name.
  ?poi ns:hasDistrict ?district.} """

In [12]:
qres=g.query(query_test)

In [17]:
for row in qres:
    print(f"{row.lat} {row.long} {row.name} {row.district}")

46.097 4.847 Aix-en-Provence TGV Ain
46.114 5.824 Bellegarde Ain
48.871 2.395 Paris-Est Paris
48.881 2.317 Paris-Nord Paris
